# LSTM

###### https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
###### https://www.nbshare.io/notebook/249468051/How-To-Code-RNN-and-LSTM-Neural-Networks-in-Python/
###### https://analyticsindiamag.com/complete-guide-to-bidirectional-lstm-with-python-codes/


In [1]:
#%pip install tensorflow
#%pip install keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
import random


In [26]:
## https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package?resource=download

random.seed(12)

df_weather = pd.read_csv("../datasets/weatherAUS.csv")

df_weather = df_weather[['Humidity9am','Humidity3pm']]
df_weather.dropna(inplace=True)
df_weather = df_weather.iloc[0:3000,]

X = df_weather[['Humidity9am']]
y = df_weather[['Humidity3pm']]

# define min max scaler
scaler = MinMaxScaler()
# transform data
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

X = pd.DataFrame(X, columns = ['Humidity9am'])
y = pd.DataFrame(y, columns = ['Humidity3pm'])

In [29]:
#model <- trainr(Y = Y[,train],
#                X = X[,train],
#                learningrate = 0.01,
#                hidden_dim = 15,
#                network_type = "rnn",
#                numepochs = 100)

## https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras import layers

model = Sequential()

#model.add(layers.Embedding(input_dim=1, output_dim=1))
#model.add(Dense(12, input_dim=8, activation='relu'))
model.add(layers.SimpleRNN(15,input_dim=1))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

model.fit(X.iloc[0:80,], X.iloc[0:80,], epochs=100, batch_size=10)

predictions = model.predict(X.iloc[81:100,])


Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_15 (SimpleRNN)   (None, 15)                255       
                                                                 
 dense_7 (Dense)             (None, 1)                 16        
                                                                 
Total params: 271
Trainable params: 271
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
8/8 [==============================] - 1s 1ms/step - loss: 0.6832 - accuracy: 0.0000e+00
Epoch 2/100
8/8 [==============================] - 0s 2ms/step - loss: 0.6792 - accuracy: 0.0000e+00
Epoch 3/100
8/8 [==============================] - 0s 2ms/step - loss: 0.6762 - accuracy: 0.0000e+00
Epoch 4/100
8/8 [==============================] - 0s 2ms/step - loss: 0.6739 - accuracy: 0.0000e+00
Epoch 5/100
8/8 [==============================

In [51]:
predictions = model.predict(X.iloc[81:101,])
predictions

X.iloc[81:101,]

print(type(predictions))

predictions_y = pd.DataFrame(predictions, columns = ['prediction'])

result = predictions_y.join(y.iloc[81:101,])

y_index = y.iloc[81:101,].reset_index()


pd.concat([predictions_y, y_index], axis=1, ignore_index=True)

<class 'numpy.ndarray'>


,0,1,2
0,0.328467,81,0.344086
1,0.534340,82,0.129032
2,0.371735,83,0.172043
3,0.360689,84,0.032258
4,0.428727,85,0.075269
5,0.371735,86,0.096774
6,0.307849,87,0.129032
7,0.417137,88,0.193548
8,0.318065,89,0.075269
9,0.417137,90,0.215054


In [1]:
## Cópia de Exemplo Predição de Séries Temporais LSTM.ipynb

import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error

In [2]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [3]:
np.random.seed(7)

#Carrega o dataset
#international airline passengers problem with time step regression framing
dataframe = read_csv('https://telescopeinstorage.blob.core.windows.net/datasets/Eletrical_Production.csv', index_col = 'DATE', engine='python')
dataframe.head(150)

,IPG2211A2N
DATE,
1939-01-01,3.3842
1939-02-01,3.4100
1939-03-01,3.4875
1939-04-01,3.5133
1939-05-01,3.5133
...,...
1951-02-01,10.2817
1951-03-01,10.2817
1951-04-01,10.2559


In [5]:
dataframeFilter =  dataframe.loc['1980-01-01':]
X = dataframeFilter[['IPG2211A2N']].values
X = X.astype('float32')

In [10]:
#Normaliza o dataset
scaler = MinMaxScaler(feature_range=(0, 1))

#scaler = StandardScaler()
X = scaler.fit_transform(X)

train_size = int(len(X) * 0.7)
test_size = len(X) - train_size

train = X[0:train_size,:]
test =  X[train_size:len(X),:]

In [11]:
# quantidade de atrasos
look_back = 3

X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

In [13]:
y_train.shape

(326,)